# 分類の予測モデル作成

In [1]:
from sklearn.datasets import load_iris
# Irisデータセットを読み込む
iris = load_iris()
X, y = iris.data, iris.target
# 先頭5行を表示
print('X:')
print(X[:5, :])
print('y:')
print(y[:5])

X:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
y:
[0 0 0 0 0]


In [2]:
# Inputを学習データとテストデータに分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(105, 4)
(45, 4)
(105,)
(45,)


## 決定木 (Decision Tree)

In [3]:
from sklearn.tree import DecisionTreeClassifier
# 決定木をインスタンス化する (気の最大の深さ3)
tree = DecisionTreeClassifier(max_depth=3)
# 学習
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [5]:
# 予測
y_pred = tree.predict(X_test)
y_pred

array([1, 1, 0, 1, 0, 1, 1, 0, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 1, 1, 2, 0,
       0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 2, 0, 1, 2, 1, 2, 1,
       2])

## ランダムフォレスト

In [6]:
from sklearn.ensemble import RandomForestClassifier
# ランダムフォレストをインスタンス化する
forest = RandomForestClassifier(n_estimators=100)
# 学習
forest.fit(X_train, y_train)
# 予測
y_pred = forest.predict(X_test)
y_pred

array([1, 1, 0, 1, 0, 1, 1, 0, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 1, 1, 2, 0,
       0, 1, 0, 1, 2, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 1, 0, 1, 2, 1, 2, 1,
       2])